## DIM0549 - Grafos : 1º Unidade, Grupo 5

### Classes abstratas contendo metodos basicos das classes *Vertice* e *Grafo*

## Item **1** : Criação do grafo a partir da lista de adjancências
A implementação do grafo que utiliza a lista de adjacência como estrutura de dados subjacente foi feita através das classes *ListaAdjacencia* e *VerticeAdjacencia*.
Aqui, contamos com o método *ler_grafo*, responsável por realizar a leitura do grafo descrito no filepath fornecido como
entrada e fazer a construção do grafo. Essa construção consiste em, para cada par de vértices lidos, instanciar os objetos
vértices correspondentes, e inserí-los no grafo via o método *adicionar_vertice*, caso já não pertençam ao grafo. Se já pertencerem, é feita a
criação de novas arestas via a função *adicionar_vizinho* encapsulada na classe *VerticeAdjacencia*. 
Maior detalhamento sobre o método de adição será fornecido na resolução do item **9**
Esta implementação da classe já conta, também, com a implementação do método de remoção de vértice ao grafo.



In [110]:
from collections import Counter
class VerticeAdjacencia:
    def __init__(self, label, vizinhos = None):
        self.label = label
        # self.predecessor = predecessor if predecessor else None
        vizinhos = vizinhos if vizinhos else []
        self.vizinhos = vizinhos
    def adicionar_vizinho(self, vizinho):
        self.vizinhos.append(vizinho)
        vizinho.vizinhos.append(self)
    def __str__(self):
        return f"Label : {self.label}"

# Nesta implementacao o grafo consistirá em um dict cujos valores sao lists (implementacoes de linkedlista=)
class ListaAdjacencia:
    def __init__(self, nome,  caminho_arquivo : str = None):
        self.lista = dict() # Ja que ordem nao importa, sempre troque o vertice a ser removido pelo ultimo vertice na lista 
        self.labels = dict()
        if caminho_arquivo:
            vertices = self.ler_grafo(caminho_arquivo=caminho_arquivo)
            for vertice in vertices:
                self.adicionar_vertice(vertice)

    def adicionar_vertice(self, novo_vertice): # O(m), tq m eh numero de vertices vizinhos ao novo_vertice JA PRESENTES NO GRAFO
        self.lista[novo_vertice] = set()
        self.labels[novo_vertice.label] = novo_vertice
        # Ver os vizinhos do vertice que vai ser adicionado
        # adicionar o novo vertice a lista de colisao de cada um de seus vizinhos 
        for vizinho in novo_vertice.vizinhos:
            self.lista[novo_vertice].add(vizinho) 
            if self.lista.get(vizinho):
                self.lista[vizinho].add(novo_vertice)

    def ler_grafo(self, caminho_arquivo : str):
        with open(caminho_arquivo, "r") as grafo_arquivo:
            linhas = grafo_arquivo.readlines()
            ordem = int(linhas[0])
            vertices = dict()
            def extrair_labels(linha: str):
                linha_split = linha.strip().split(",")
                return linha_split[0], linha_split[1]

            for linha in linhas[1:]:
                label_1, label_2 = extrair_labels(linha)
                # print(f"Labels: {label_1} e {label_2}")
                # Checa se os vertices recebidos na linha ja existem
                vert_1, vert_2 = vertices.get(label_1, None), vertices.get(label_2, None)
                if vert_1 and vert_2:
                    vert_1.adicionar_vizinho(vert_2)
                elif vert_1:
                    vert_2 = VerticeAdjacencia(label=label_2)
                    vert_1.adicionar_vizinho(vert_2)
                    vertices[label_2] = vert_2
                elif vert_2:
                    vert_1 = VerticeAdjacencia(label=label_1, vizinhos=[vert_2])
                    vertices[label_1] = vert_1
                else:
                    vert_2 = VerticeAdjacencia(label=label_2)
                    vert_1 = VerticeAdjacencia(label=label_1, vizinhos=[vert_2])
                    vertices[label_1] = vert_1
                    vertices[label_2] = vert_2
            return vertices.values()
    def remover_vertice(self, vertice_a_ser_removido):
        if vertice_a_ser_removido not in self.lista:
            return "Vertice nao pertence ao grafo"
        self.lista.pop(vertice_a_ser_removido)  # O(1)
        for vizinho in vertice_a_ser_removido.vizinhos: # O(m)
            del self.lista[vizinho][vertice_a_ser_removido] # O(1)

    def get_vertices(self):
        return self.lista.keys()

    def get_vertice(self,label):
        return self.labels[label]
    

    def get_adjacentes(self, label_vertice):
        vertice = self.labels[label_vertice]
        return set(vertice.vizinhos)

    def __str__(self):
        if len(self.lista) ==0:
            return "<ListaAdjacencia Vazio>"
        strings = [f"{vertice.label} -> {list(map(lambda vizinho : vizinho.label, self.lista[vertice]))}" for vertice in self.lista.keys()]
        return '\n'.join(strings)


grafo = ListaAdjacencia("Grafo 1",caminho_arquivo="./data/GRAFO_1.txt")
print(grafo)


a -> ['b']
b -> ['c', 'f', 'e', 'd']
c -> ['b', 'e', 'd', 'i']
d -> ['b', 'c', 'h']
e -> ['b', 'c']
f -> ['b']
i -> ['c']
h -> ['d']


## Item **2** : Implementação do Grafo usando *Matriz de adjacência*
A implementação usando matriz de adjacência tem a mesma estrutura do item anterior, é feita a leitura via método *ler_grafo* e adição dos vértices.

In [111]:
from collections import Counter
from functools import reduce

# Implementacao da classe vertice para de grafos utilizando matriz de adjacencia 
class VerticeMatriz:
    def __init__(self, label : str = "", vizinhos = None):
        self.label = label 
        self.indice = None
        # O Counter associa um objeto a um contador numérico via hash table, aqui será usado para contar o número de
        # arestas entre este vértice e seu respectivo vizinho
        self.vizinhos = Counter() 
        if vizinhos:
            for vizinho in vizinhos:
                self.adicionar_vizinho(vizinho)

    def adicionar_vizinho(self, vertice_vizinho):
        self.vizinhos[vertice_vizinho] += 1
        vertice_vizinho.vizinhos[self] += 1

class MatrizAdjacencia:
    def __init__(self, nome : str , caminho_arquivo : str = None): # O(n^2)
        self.nome = nome # Nome do grafo, pra proposito de impressao
        self.indices = dict() # hash table contendo os vertices associados a cada indice da matriz, para agilizar consultas
        self.matriz = [] 
        self.labels = dict() # Hash table para armazenar os vértices associadas a cada label
        if not caminho_arquivo:
            return 
        for vertice in self.ler_grafo(caminho_arquivo):
            self.adicionar_vertice(vertice)

    def ler_grafo(self, caminho_arquivo : str):
        with open(caminho_arquivo, "r") as grafo_arquivo:
            linhas = grafo_arquivo.readlines()
            ordem = int(linhas[0])
            vertices = dict()
            def extrair_labels(linha: str):
                linha_split = linha.strip().split(",")
                return linha_split[0], linha_split[1]

            for linha in linhas[1:]:
                label_1, label_2 = extrair_labels(linha)

                # Checa se os vertices recebidos na linha ja existem
                vert_1, vert_2 = vertices.get(label_1, None), vertices.get(label_2, None)
                if vert_1 and vert_2:
                    vert_1.adicionar_vizinho(vert_2)
                elif vert_1:
                    vert_2 = VerticeMatriz(label=label_2)
                    vert_1.adicionar_vizinho(vert_2)
                    vertices[label_2] = vert_2
                elif vert_2:
                    vert_1 = VerticeMatriz(label=label_1, vizinhos=[vert_2])
                    vertices[label_1] = vert_1
                else:
                    vert_2 = VerticeMatriz(label=label_2)
                    vert_1 = VerticeMatriz(label=label_1, vizinhos=[vert_2])
                    vertices[label_1] = vert_1
                    vertices[label_2] = vert_2
            return vertices.values()


    def adicionar_vertice(self, vertice : VerticeMatriz): # O(n)
        # Adiciona o novo vertice as linhas anteriores
        self.labels[vertice.label] = vertice
        if len(self.matriz) == 0:
            vertice.indice = 0
            self.matriz.append(list([vertice.vizinhos[vertice]]))

            self.indices[0] = vertice
            return

        # Linha que será adcioanada à matriz
        nova_linha = []
        # Adiciona o índice do novo vértice à hash table
        self.indices[len(self.matriz)] = vertice # O(1)
        vertice.indice = len(self.matriz) # O(1)
        for indice, linha in enumerate(self.matriz): # O(n)
            vertice_atual = self.indices[indice] # O(1)
            # Recupera atribui à posiçao da linha atual o número de arestas entre o vértice adicionado e o correspondente à linha`
            quantidade = vertice.vizinhos.get(vertice_atual, 0) # O(1)
            linha.append(quantidade) # O(1)
            nova_linha.append(quantidade) #O(1)

        # Atualiza a nova linha para o caso do vértice adicionado apresentar laços
        quantidade_arestas_proprias = vertice.vizinhos.get(vertice, 0) # O(1)
        nova_linha.append(quantidade_arestas_proprias) # O(1)
        self.matriz.append(nova_linha) #O(1)

    def get_vertices(self):
        return self.indices.values()

    def get_adjacentes(self, label_vertice):
        vertice = self.labels[label_vertice]
        indice_na_matriz = vertice.indice
        linha = self.matriz[indice_na_matriz]
        vizinhos_map = map(lambda i : self.indices[i], filter(lambda indice : linha[indice] > 0, range(0, len(linha))))
        return set(vizinhos_map)

    def get_vertice(self, label_vertice):
        return self.labels[label_vertice]

    def remover_vertice(self, vertice): # O(n + n*n) = O(n^2)
        if vertice not in self.indices.values():
            raise ValueError("Vértice não faz parte do grafo") # O(1)

        # Remove da matriz a linha referente ao vértice removido
        indice_a_remover = vertice.indice
        self.matriz.pop(indice_a_remover)

        # Atualiza a tabela de índices para refletir as novas posições
        for j in range(indice_a_remover + 1,len(self.indices)): # O(n)
            self.indices[j - 1] = self.indices[j]
            self.indices[j - 1].indice = j - 1
            self.indices.pop(j)
        for i, linha in enumerate(self.matriz): # O(n - 1)
            if linha[indice_a_remover] != 0:    # O(1)
                self.indices[i].vizinhos.pop(vertice) # O(1)
            linha.pop(indice_a_remover) # O(n)

    def __str__(self):
        return str(self.matriz)

In [112]:
grafo = MatrizAdjacencia("Grafo - 1", "./data/GRAFO_1.txt")
print(grafo)

[[0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 1, 1, 1, 1, 0, 0], [0, 1, 0, 1, 1, 0, 1, 0], [0, 1, 1, 0, 0, 0, 0, 1], [0, 1, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0]]


## Item **14** - Busca em Profundidade, com determinação de arestas de retorno, a partir de um vértice em específico

In [ ]:
# TODO : Retornar árvore de profundidade e checar se precisa fazer o percurso na ordem
def depth_first_search(grafo, label_vertice_inicial, ordenado : bool = None):
    # É necessário manter estado das arestas visitadas. Pra representar uma aresta, usaremos tuplas contendo referências aos vertices 
    pilha = list()
    vertices_visitados = set()
    vertices_explorados = set()
    vertice_inicial = grafo.get_vertice(label_vertice_inicial)
    vertices_visitados.add(vertice_inicial)
    arestas_visitadas = set()
    arestas_de_retorno = set()

    print("## Percurso em profundidade") 
    print("Vertice inicial: ", vertice_inicial.label)
    pilha.append(vertice_inicial)
    vizinhos_vertices = dict()
    while len(pilha) > 0:
        u = pilha[-1]
        if u not in vizinhos_vertices:
            if ordenado:
                # TODO : Refatorar
                labels_ordenadas = sorted([i.label for i in grafo.get_adjacentes(u.label)], reverse=True)
                vizinhos_vertices[u] = [grafo.get_vertice(v) for v in labels_ordenadas]
            else:
                vizinhos_vertices[u] = grafo.get_adjacentes(u.label)
        
        v = vizinhos_vertices[u].pop() if len(vizinhos_vertices[u]) > 0 else None
        if not v:
            pilha.pop()
            vertices_explorados.add(v)
        elif v not in vertices_visitados:
            vertices_visitados.add(v)
            print(f"Vertice visitado: {v.label}, predecessor : {u.label}", )
            pilha.append(v)
            arestas_visitadas.add((u,v))
        else:
            if v not in vertices_explorados and pilha[-2] != v: # Checa se é adjaccente na pilha
                if (v,u) not in arestas_visitadas:
                    arestas_visitadas.add((u,v))
                    arestas_de_retorno.add((u,v))

    # Printa arestas de retorno
    arestas_de_retorno = map(lambda aresta : (aresta[0].label, aresta[1].label), arestas_de_retorno)
    print(f"Arestas de retorno: {list(arestas_de_retorno)}")

In [114]:
# Funciona
grafo1 = MatrizAdjacencia("Grafo Matriz", "./data/GRAFO_1.txt")
depth_first_search(grafo=grafo1, label_vertice_inicial="a", ordenado=True)

grafo1 = ListaAdjacencia("Grafo Lista", "./data/GRAFO_1.txt")
depth_first_search(grafo=grafo1, label_vertice_inicial="a", ordenado=True)


## Percurso em profundidade
Vertice inicial:  a
Vizinhos ordenados de : ['b']
Vertice visitado: b, predecessor : a
Vizinhos ordenados de : ['f', 'e', 'd', 'c', 'a']
Vertice visitado: c, predecessor : b
Vizinhos ordenados de : ['i', 'e', 'd', 'b']
Vertice visitado: d, predecessor : c
Vizinhos ordenados de : ['h', 'c', 'b']
Vertice visitado: h, predecessor : d
Vizinhos ordenados de : ['d']
Vertice visitado: e, predecessor : c
Vizinhos ordenados de : ['c', 'b']
Vertice visitado: i, predecessor : c
Vizinhos ordenados de : ['c']
Vertice visitado: f, predecessor : b
Vizinhos ordenados de : ['b']
Arestas de retorno: [('d', 'b'), ('e', 'b')]
## Percurso em profundidade
Vertice inicial:  a
Vizinhos ordenados de : ['b']
Vertice visitado: b, predecessor : a
Vizinhos ordenados de : ['f', 'e', 'd', 'c']
Vertice visitado: c, predecessor : b
Vizinhos ordenados de : ['i', 'e', 'd', 'b']
Vertice visitado: d, predecessor : c
Vizinhos ordenados de : ['h', 'c', 'b']
Vertice visitado: h, predecessor : d
Vi

## Item **13** - Busca em Largura, a partir de um vértice específico.

In [115]:
from collections import deque
def width_first_search(grafo, label_vertice_inicial):
    fila = deque()
    vertices_marcados = set()
    arestas_visitadas = set()
    vertice_inicial = grafo.get_vertice(label_vertice_inicial)
    vertices_marcados.add(vertice_inicial)
    print("## Percurso em Largura")
    print("Vertice inicial: ", vertice_inicial.label)
    fila.append(vertice_inicial)
    vizinhos_vertices = dict()
    while len(fila) > 0:
        u = fila.popleft()
        # Se os vertices adjacentes de u nao estiverem em nosso dict que os armazena, os insere
        vizinhos = grafo.get_adjacentes(u.label)
        v = vizinhos.pop() if len(vizinhos) > 0 else None
        while v: # O(n) -> n : numero de vizinhos do vertice u
            if v not in vertices_marcados:
                vertices_marcados.add(v)
                fila.append(v)
                print(f"Vertice visitado: {v.label}; Predecessor : {u.label}",)
            v = vizinhos.pop() if len(vizinhos) > 0 else None

In [116]:
grafo_matriz = ListaAdjacencia("Grafo Matriz", "./data/GRAFO_1.txt")
print(grafo_matriz)
width_first_search(grafo_matriz, "a")

a -> ['b']
b -> ['d', 'e', 'c', 'f']
c -> ['d', 'b', 'e', 'i']
d -> ['h', 'b', 'c']
e -> ['b', 'c']
f -> ['b']
i -> ['c']
h -> ['d']
## Percurso em Largura
Vertice inicial:  a
Vertice visitado: b; Predecessor : a
Vertice visitado: d; Predecessor : b
Vertice visitado: e; Predecessor : b
Vertice visitado: c; Predecessor : b
Vertice visitado: f; Predecessor : b
Vertice visitado: h; Predecessor : d
Vertice visitado: i; Predecessor : c
